In [1]:
#importing Libraries and data

import pandas as pd
data = pd.read_csv("data.csv", header = None)
dataset = data

In [2]:
#formatting data

df = dataset.T.apply(lambda x: x.dropna().tolist()).tolist()

In [3]:
#function to implement the Apriori algorithm for frequent itemset generation.

#For run-time meaurement

import time
start_time = time.time()

#importing the required libraries

from numpy import *

k=1
#Function to create one frequent itemset

def createI1(df):
    I1 = []               #I1 =one itemset list
    for rows in df:
        for elements in rows:
            if not [elements] in I1:
                I1.append([elements])             
    I1.sort()
    return list(map(frozenset, I1))

# Function to prune any itemset with given minimum support

def candidatePrune(data,candidateList, minSupport):
    subsetCount = {}
    for rows in data:
        for candidate in candidateList:
            if candidate.issubset(rows):
                if not candidate in subsetCount:
                    subsetCount[candidate] = 1   
                else:
                    subsetCount[candidate] +=1    
    prunedList = []
    supportData = {}
    
    for key in subsetCount:
        support = subsetCount[key]/float(len(data))
        if support >= minSupport:
            prunedList.insert(0,key)
        supportData[key] = support
    return prunedList, supportData

#Function for candidate itemset generation

def candidateGen(freqItemset, itemsetSize):
    candidateItemset = []
    lengthFI = len(freqItemset)
    for i in range(lengthFI):
        for j in range(i+1, lengthFI):
            L1 = list(freqItemset[i])[: k-2]
            L2 = list(freqItemset[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                candidateItemset.append(freqItemset[i] | freqItemset[j])
    return candidateItemset

################################################################################
#Main apriori algorithm

def apriori(dataset, minSupport = 0.01):
    I1 = createI1(dataset)
    data = list(map(set, dataset))
    prunedList1, supportData = candidatePrune(data, I1, minSupport)
    prunedList= [prunedList1]
    k=2
    while (len(prunedList[k-2]) > 0):
        canItem = candidateGen(prunedList[k-2],k)
        canPruned, canSupport = candidatePrune(data, canItem, minSupport)
        supportData.update(canSupport)
        prunedList.append(canPruned)
        k=k+1
        
    return prunedList, supportData


In [4]:
#running the apriori algorithm
freqItemset, supportData = apriori(df)

In [5]:
length =0
for i in range(0,len(freqItemset)):
    length = length + len(freqItemset[i])
length

159

In [6]:
#getting the run-time estimates
print("---%s seconds ---" % (time.time()-start_time))

---187.8123676776886 seconds ---


In [7]:
#function to implement the simple randomized algorithm for frequent itemset generation.

# for selecting dataset I have used random sampling using the percentage of data 
p = 0.10      #select percentage of samples
n = int(p*len(dataset)) # total number of samples
import random
# randomly select given number of samples
df1 = random.choices(df,k =n)

In [8]:

#For run-time meaurement
import time
start_time = time.time()

#importing the required libraries
from numpy import *

k=1
#Function to create one frequent itemset
def createI1(df1):
    I1 = []               #I1 =one itemset list
    for rows in df1:
        for elements in rows:
            if not [elements] in I1:
                I1.append([elements])             
    I1.sort()
    return list(map(frozenset, I1))
# Function to prune any itemset with given minimum support
def candidatePrune(data,candidateList, minSupport):
    subsetCount = {}
    for rows in data:
        for candidate in candidateList:
            if candidate.issubset(rows):
                if not candidate in subsetCount:
                    subsetCount[candidate] = 1   
                else:
                    subsetCount[candidate] +=1    
    prunedList = []
    supportData = {}
    
    for key in subsetCount:
        support = subsetCount[key]/float(len(data))
        if support >= minSupport:
            prunedList.insert(0,key)
        supportData[key] = support
    return prunedList, supportData
#Function for candidate itemset generation
def candidateGen(freqItemset, itemsetSize):
    candidateItemset = []
    lengthFI = len(freqItemset)
    for i in range(lengthFI):
        for j in range(i+1, lengthFI):
            L1 = list(freqItemset[i])[: k-2]
            L2 = list(freqItemset[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                candidateItemset.append(freqItemset[i] | freqItemset[j])
    return candidateItemset

###########################################
#Main apriori algorithm

def apriori(dataset, minSupport = 0.01):
    I1 = createI1(dataset)
    data = list(map(set, dataset))
    prunedList1, supportData = candidatePrune(data, I1, minSupport)
    prunedList= [prunedList1]
    k=2
    while (len(prunedList[k-2]) > 0):
        canItem = candidateGen(prunedList[k-2],k)
        canPruned, canSupport = candidatePrune(data, canItem, minSupport)
        supportData.update(canSupport)
        prunedList.append(canPruned)
        k=k+1
        
    return prunedList, supportData

In [9]:
freqItemset, supportData = apriori(df1)

In [10]:
length =0
for i in range(0,len(freqItemset)):
    length = length + len(freqItemset[i])
length

162

In [11]:
print("---%s seconds ---" % (time.time()-start_time))

---20.533121347427368 seconds ---


In [12]:
#Function to create rules for associaiton analysis
def ruleGeneration (freqItemset, supportData, minConf =0.05):
    ruleList = []
    k = len(freqItemset)
    for i in range (1, k):
        for freqSet in freqItemset[i]:
            H1 = [frozenset ([item]) for item in freqSet]
            if(i>1):
                conseqRules(freqSet, H1, supportData, ruleList, minConf)
            else:
                calculateConfidence(freqSet, H1, supportData, ruleList, minConf)
    return ruleList
def calculateConfidence(freqSet, H, supportData, ruleList1, minConf=0.05):
    prunedH = []
    for consequent in H:
        confidence = supportData[freqSet] / supportData[freqSet-consequent]
        if confidence >= minConf:
            print (freqSet-consequent, '-->', consequent, 'confVal=', round(confidence,4),
                   'support = ', round(supportData[consequent],3), 'lift = ', round(confidence/supportData[consequent],3))
            ruleList1.append((freqSet-consequent, consequent, confidence))
            prunedH.append(consequent)
    return prunedH
def conseqRules(freqSet, H, supportData, ruleList1, minConf=0.05):
    k = len(freqSet)
    m = len(H[0])
    if (k > (m+1)):
        Hmplus1 = candidateGen(H, m+1)
        Hmplus1 = calculateConfidence(freqSet, Hmplus1, supportData, ruleList1, minConf)
        if (len(Hmplus1)>1):
            conseqRules(freqSet, Hmplus1, supportData, ruleList1, minConf)

In [13]:
rulesGenerated = ruleGeneration(freqItemset, supportData, minConf=0.05)

frozenset({14099.0}) --> frozenset({40.0}) confVal= 0.6118 support =  0.582 lift =  1.051
frozenset({287.0}) --> frozenset({39.0}) confVal= 0.9262 support =  0.179 lift =  5.185
frozenset({39.0}) --> frozenset({287.0}) confVal= 0.0717 support =  0.014 lift =  5.185
frozenset({80.0}) --> frozenset({40.0}) confVal= 0.6813 support =  0.582 lift =  1.17
frozenset({80.0}) --> frozenset({49.0}) confVal= 0.5625 support =  0.481 lift =  1.17
frozenset({256.0}) --> frozenset({40.0}) confVal= 0.7597 support =  0.582 lift =  1.305
frozenset({39.0}) --> frozenset({38.0}) confVal= 0.0698 support =  0.013 lift =  5.401
frozenset({38.0}) --> frozenset({39.0}) confVal= 0.9649 support =  0.179 lift =  5.401
frozenset({12926.0}) --> frozenset({40.0}) confVal= 0.6824 support =  0.582 lift =  1.172
frozenset({42.0}) --> frozenset({226.0}) confVal= 0.0618 support =  0.039 lift =  1.592
frozenset({226.0}) --> frozenset({42.0}) confVal= 0.2749 support =  0.173 lift =  1.592
frozenset({256.0}) --> frozenset({

In [14]:
len(rulesGenerated)

176